In [26]:
import numpy as np
import pandas as pd

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#stats
import statsmodels.api as sm
from statsmodels.api import tsa



In [2]:
air_traffic = pd.read_csv('/content/drive/MyDrive/projects/airline_TSA/air_traffic.csv')
air_traffic.head()

,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12
3,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07
4,2003,6,"52,209,516","5,411,504","57,621,020","798,351","58,225","856,576","44,492,972","13,918,185","58,411,157","56,555,517","17,191,579","73,747,096",78.67,80.96,79.20


In [3]:
#lets combine year and month and add a day as 1st of each month to have a full date column
air_traffic['Date']= pd.to_datetime(air_traffic.assign(Day=1).loc[:, ['Year', 'Month', 'Day']])
air_traffic.head()


,Year,Month,Dom_Pax,Int_Pax,Pax,Dom_Flt,Int_Flt,Flt,Dom_RPM,Int_RPM,RPM,Dom_ASM,Int_ASM,ASM,Dom_LF,Int_LF,LF,Date
0,2003,1,"43,032,450","4,905,830","47,938,280","785,160","57,667","842,827","36,211,422","12,885,980","49,097,402","56,191,300","17,968,572","74,159,872",64.44,71.71,66.20,2003-01-01
1,2003,2,"41,166,780","4,245,366","45,412,146","690,351","51,259","741,610","34,148,439","10,715,468","44,863,907","50,088,434","15,587,880","65,676,314",68.18,68.74,68.31,2003-02-01
2,2003,3,"49,992,700","5,008,613","55,001,313","797,194","58,926","856,120","41,774,564","12,567,068","54,341,633","57,592,901","17,753,174","75,346,075",72.53,70.79,72.12,2003-03-01
3,2003,5,"49,152,352","4,610,834","53,763,186","789,397","55,265","844,662","41,001,934","11,575,026","52,576,960","55,349,897","15,629,821","70,979,718",74.08,74.06,74.07,2003-05-01
4,2003,6,"52,209,516","5,411,504","57,621,020","798,351","58,225","856,576","44,492,972","13,918,185","58,411,157","56,555,517","17,191,579","73,747,096",78.67,80.96,79.20,2003-06-01


In [4]:
#since we are only focusing overall flight stats, we will remove all domestic and international individual columns
air_traffic_removed= air_traffic.drop(columns=['Dom_Pax','Int_Pax','Dom_Flt','Int_Flt','Dom_RPM','Int_RPM','Dom_ASM','Int_ASM','Dom_ASM','Year','Month','Dom_LF','Int_LF','LF','ASM'])

In [5]:
#let's rename the columns for better understanding
air_traffic_renamed= air_traffic_removed.rename(columns={'Pax':'Passengers','Flt':'Flights','RPM':'Revenue Per Mile'})
air_traffic_renamed.head()

,Passengers,Flights,Revenue Per Mile,Date
0,"47,938,280","842,827","49,097,402",2003-01-01
1,"45,412,146","741,610","44,863,907",2003-02-01
2,"55,001,313","856,120","54,341,633",2003-03-01
3,"53,763,186","844,662","52,576,960",2003-05-01
4,"57,621,020","856,576","58,411,157",2003-06-01


In [6]:
# EDA
air_traffic_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Passengers        233 non-null    object        
 1   Flights           233 non-null    object        
 2   Revenue Per Mile  233 non-null    object        
 3   Date              243 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 7.7+ KB


In [7]:
# let's update the index of dataframe to date,
# To perform any TSA, keeping date to index enables a lot of flexibility
air_traffic_renamed= air_traffic_renamed.set_index('Date')
air_traffic_renamed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 243 entries, 2003-01-01 to 2023-09-01
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Passengers        233 non-null    object
 1   Flights           233 non-null    object
 2   Revenue Per Mile  233 non-null    object
dtypes: object(3)
memory usage: 7.6+ KB


In [8]:
# let's convert the values to appropriate datatype
air_traffic_renamed['Passengers'] = air_traffic_renamed['Passengers'].str.replace(',', '', regex=False).astype('Int64')
air_traffic_renamed['Flights'] = air_traffic_renamed['Flights'].str.replace(',', '', regex=False).astype('Int64')
air_traffic_renamed['Revenue Per Mile'] = air_traffic_renamed['Revenue Per Mile'].str.replace(',', '', regex=False).astype('Int64')




In [9]:
air_traffic_renamed.head()

,Passengers,Flights,Revenue Per Mile
Date,,,
2003-01-01,47938280,842827,49097402
2003-02-01,45412146,741610,44863907
2003-03-01,55001313,856120,54341633
2003-05-01,53763186,844662,52576960
2003-06-01,57621020,856576,58411157


In [10]:
# we can see that there are null rows in our data
# let's visualize how it looks at the begining
fig = px.line(
    air_traffic_renamed,
    x=air_traffic_renamed.index,
    y=air_traffic_renamed.columns,
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Traffic Data from 2003 to 2023'
)


fig.update_layout(
    yaxis_title=' air traffic data',
    title='Traffic Data from 2003 to 2023'

)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the

In [11]:
# Before doing any update to the nulls let's examine if there are any missing dates
# let's verify the total number of months between 2003-01 and 2023-09
# how we will do it is see the total number of months between those two range and compare with our data
first_day = air_traffic_renamed.index.min()
last_day= air_traffic_renamed.index.max()
total_months = (last_day.year - first_day.year)*12 + (last_day.month-first_day.month) + 1

air_traffic_renamed.shape[0]== total_months

False

In [12]:
# since the dates are not complete, we will first fill in the date date indexes
# let's get the date range from first day to last day
full_dates_range= pd.date_range(start=first_day,end=last_day,freq='MS')

In [13]:
# we are missing the above dates in our dataset
# Now let's add these dates to our dateindex dataset by reindexing

air_traffic_dates_updated= air_traffic_renamed.reindex(full_dates_range)

air_traffic_dates_updated.shape

(249, 3)

In [14]:
# dates have been added , let's check the data
air_traffic_dates_updated.isna().sum()

,0
Passengers,16
Flights,16
Revenue Per Mile,16


In [15]:
# To impute the missing values, we will use one of the following imputation technique
# ffill - fill the values based on the last available data
# bfill - fill the values based on the next available data
# interpolation - fill the values based on average of first two available data forward and backward

fig = px.line(
    air_traffic_dates_updated,
    x=air_traffic_dates_updated.index,
    y=air_traffic_dates_updated['Revenue Per Mile'],
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Revenue per mile from 2003 to 2023'
)


fig.update_layout(
    yaxis_title=' air traffic data',

)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [16]:
# in those missing values, we will try to see how the graph looks with imputation using ffill and bfill
# we can choose the best fill after seeing the graph which looks more consistent
air_traffic_impute_test = air_traffic_dates_updated.copy()
air_traffic_impute_test['bfill']= air_traffic_impute_test['Revenue Per Mile'].bfill()
air_traffic_impute_test['ffill']= air_traffic_impute_test['Revenue Per Mile'].ffill()


In [17]:
# let's compare both fills in graph
fig = px.line(
    air_traffic_impute_test,
    x=air_traffic_impute_test.index,
    y=['Revenue Per Mile', 'bfill', 'ffill'],
    labels={'x': 'Date', 'value': 'Air Traffic Data'},
    title='Revenue per mile from 2003 to 2023'
)

fig.update_layout(
    yaxis_title='Air Traffic Data',
)

fig.update_xaxes(rangeslider_visible=True)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [18]:
# Looking at the graph the better would be ffill , seeing the consistency
air_traffic_final = air_traffic_dates_updated.fillna(method='ffill')
air_traffic_final.isna().sum()

<ipython-input-18-8d673472d6c1>:2: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,0
Passengers,0
Flights,0
Revenue Per Mile,0


In [19]:
# comparing revenue of each month to the overall average revenue

# monthly mean of individual months over the whole time period
monthly_mean = air_traffic_final.groupby(air_traffic_final.index.month_name())['Revenue Per Mile'].mean()

# relative deviation from whole mean
monthly_mean_diff = (monthly_mean - monthly_mean.mean())/monthly_mean

# monthly names in order
monthly_names = pd.date_range(start='2000-01',freq='MS', periods=12).month_name()

monthly_mean_diff = monthly_mean_diff.loc[monthly_names,]

monthly_mean_diff.to_frame().T

,January,February,March,April,May,June,July,August,September,October,November,December
Revenue Per Mile,-0.114549,-0.201075,0.016827,0.007353,0.017342,0.087572,0.141732,0.108232,-0.053882,-0.013855,-0.080244,-0.015628


In [20]:
# To actually visualize the rate of changes, we can plot a graph
fig = px.bar(monthly_mean_diff)

fig.update_layout(
    title ='Monthly deviation from mean revenue'
)
fig.show()

In [21]:
# From the data and the graph, it is clearly visible that June,July,August are the most busy months
# while january and febraury have the lowest air travel

In [22]:
# A fundamental step in Time series analysis is trend-seasonal decomposition
  # trend
  # seasonal
  # residual
# there are numerous ways to do the break down, but the most standard is seasonal_decompose function from the tsa module of statsmodels


In [23]:
# There was a covid restriction starting from 2020, so we will take data before 2020
Rev_Per_Mile = pd.DataFrame(air_traffic_final.loc[air_traffic_final.index<'2020-01-01','Revenue Per Mile'])
decomposition = tsa.seasonal_decompose(Rev_Per_Mile,model='additive',period=12)


In [24]:
Rev_Per_Mile["Trend"]= decomposition.trend
Rev_Per_Mile['Seasonal']= decomposition.seasonal
Rev_Per_Mile['Residual']= decomposition.resid
Rev_Per_Mile.head(10)

,Revenue Per Mile,Trend,Seasonal,Residual
2003-01-01,49097402,NaN,-7.228978e+06,NaN
2003-02-01,44863907,NaN,-1.225336e+07,NaN
2003-03-01,54341633,NaN,2.250784e+06,NaN
2003-04-01,54341633,NaN,1.200530e+06,NaN
2003-05-01,52576960,NaN,2.669399e+06,NaN
2003-06-01,58411157,NaN,7.598152e+06,NaN
2003-07-01,63838718,5.464377e+07,1.145216e+07,-2.257215e+06
2003-08-01,62888957,5.499366e+07,8.488630e+06,-5.933350e+05
2003-09-01,50390709,5.552558e+07,-4.535707e+06,-5.991687e+05
2003-10-01,54927858,5.605449e+07,-1.286264e+06,1.596302e+05


In [27]:
cols = ["Trend","Seasonal","Residual"]
fig = make_subplots(rows=3,cols=1,subplot_titles=cols)

for i, col in enumerate(cols):
  fig.add_trace(
      go.Scatter(x=Rev_Per_Mile.index,y=Rev_Per_Mile[col]),
      row=i+1,
      col=1

  )
fig.update_layout(height= 1000, width=1200, showlegend=False)
fig.show()


**Forecasting**

Based on the business context, we can focus on two alternative goals:

*   single-step (short-term) forecasts, or
*   multi-step (long-term) forecasts.

Since we are interested in how the air travel industry will recover from the 2001 shock, we are interested in the latter. Nonetheless, the focus here determines the way we fit and evaluate our models.

Many standard forecasting techniques work best on stationary series, one where the statistical properties of the series (such as mean and variance) don't change over time. Often, the residual after the decomposition is stationary however we did see some remaining seasonality for our time series.

We can address the latter issue by transforming the data (taking the logarithm or differencing) or by using more advanced decomposition techniques. Here, we show the effect of seasonal differencing when we look to model the year-over-yeas change, instead of the original data:



In [40]:
air_traffic_final_rev = pd.DataFrame(air_traffic_final.loc[air_traffic_final.index<'2020-01-01','Revenue Per Mile'])
air_traffic_final_rev['Seasonal difference']= air_traffic_final_rev.diff(12)


In [41]:
air_traffic_final_rev[['Seasonal difference','Revenue Per Mile']].head(16)

,Seasonal difference,Revenue Per Mile
2003-01-01,<NA>,49097402
2003-02-01,<NA>,44863907
2003-03-01,<NA>,54341633
2003-04-01,<NA>,54341633
2003-05-01,<NA>,52576960
2003-06-01,<NA>,58411157
2003-07-01,<NA>,63838718
2003-08-01,<NA>,62888957
2003-09-01,<NA>,50390709
2003-10-01,<NA>,54927858


The seasonal difference is visualized below:

In [42]:
fig = px.line(air_traffic_final_rev,x= air_traffic_final_rev.index,y= 'Seasonal difference')
fig.update_layout(
    yaxis_title='seasonal difference',
    xaxis_title= 'Date',
    title = '12 month difference data'
)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Now, apart from the last year, it looks like we got stationary data:
* there is no clear trend in the new series,
* the variance is relatively constant, and
* there is no seasonality but some multiyear cycles can be clearly spotted (a cyclical pattern is characterized by rises and falls of uneven frequency).

Note that this corresponds to the fast vs slow growth periods that we observed initially.

Our forecasting work will be based on predicting this differenced series instead of the original one.
Note: the original revenue series can be restored by using the first 12-month of revenue values and the seasonal differences by recursively adding the differences to the so-far restored values. We will do this once our forecast is ready.

**Splitting the Series for Evaluation**

Our main goal is to explore models which can reliably forecast future travel revenue. To get a fair evaluation of such models, we will use some of the available data for fitting the model and some for evaluating our forecast. This ensures that calculated metrics reflect the models performance on unseen, future data.

For our case:
* we will use observations less than 2014-01-01 as the training set to infer the model parameters, and
* subsequent records as test data for evaluation.



In [46]:
train = air_traffic_final_rev.loc[air_traffic_final_rev.index<'2014-01-01','Seasonal difference'].dropna()
test = air_traffic_final_rev.loc[air_traffic_final_rev.index>='2014-01-01','Seasonal difference']

In [47]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name="Train"))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name="Test"))
fig.update_layout( yaxis_title="Difference (billions)", xaxis_title="Date", title="Change in Revenue Passenger Mites over Prior Year" )
fig.show()

**Baseline Forecasts and Evaluation**

There are a number of ways to make simple forecasts which can act as baselines for putting more complex models in context:
* for short-term forecasts, we can predict the last available value (or a rolling average of previous values);
* for long-term forecasts of stationary series, we can predict the mean over the training set;
* for non-stationary data, we can model the trend (e.g., using linear rParession) to forPrast future trend and add the seasonality to obtain a forecast for the whole series.— Code + Markdown

Since our differenced series is stationary, we will use the mean (over the training set) as our baseline.

In [48]:
full_index = pd.concat([train, test]).index
base_line = np.full(full_index.shape, np.mean(train))
predictions = pd.Series(data=base_line, index=full_index)
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name="Train"))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name="Test"))
fig.add_trace(go.Scatter(x=predictions.index, y=predictions, mode='lines', name="Mean Prediction"))
fig.update_layout(
    yaxis_title="Difference (billions)",
    xaxis_title="Date",
    title="Change in Revenue Passenger Miles over Prior Year"
)
fig.show()

As with any regression model, we can use the mean absolute error or root mean squared error to evaluate our models (or apply other use-case specific measure or performance).

Both previous metrics are heavily influenced by the scale of the data. To put these numbers into perspective, we can look at what percentage (on average) our prediction is off by the original value. This metric is called the mean absolute percentage error (MAPE) and going forward, we will use this metric to compare our models quantitatively.

In [51]:
def mean_absolute_percentage_error(true_values,predicted_values):
  """
  Calculate the mean absolute percentage error.
  Find the prediction error and devide by the true value, then average.
  """

  error = true_values - predicted_values
  absolute_percentage_error = np.abs(error/true_values)
  mape = absolute_percentage_error.mean() * 100
  return mape
train_mape = mean_absolute_percentage_error(train, predictions[train.index])
test_mape = mean_absolute_percentage_error(test, predictions[test.index])
print(f"Train MAPE on the difference: {round(train_mape, 2)}%")
print(f"Test MAPE on the difference: {round(test_mape, 2)}%")



Train MAPE on the difference: 136.6%
Test MAPE on the difference: 67.01%
